In [1]:
from glob import glob
import json
import matplotlib.pyplot as plt
import numpy as np

In [3]:
cv_staff = glob('../archive/cv_staff/*.json')
print(len(cv_staff))

9422


In [4]:
import pandas as pd
anime_data = pd.read_csv('../archive/anime.csv')
anime_data.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [9]:
anime_ids = anime_data['anime_id'].values
types = anime_data['type'].values
anime_type_dict = {}
print(anime_ids)
for a,t in zip(anime_ids, types):
    anime_type_dict[a] = t

[32281  5114 28977 ...  5621  6133 26081]


In [21]:
from tqdm.notebook import tqdm
import collections
name_count = collections.defaultdict(int)
cv_count = collections.defaultdict(int)
valid_type = {'TV', 'OVA', 'Movie'}
anime_cv_dict = {}
for cs in tqdm(cv_staff):
    name = cs.split('/')[-1].split('.json')[0]
    anime_id = int(name.split('_')[0])
    if anime_id not in anime_type_dict:
        continue
    if anime_type_dict[anime_id] not in valid_type:
        continue
    d = json.load(open(cs))
    cv_info = d['cv']
    anime_cv_dict[name] = {'main':[], 'sup':[], 'staff':[]}
    for c in cv_info:
        if c['chara_type'] == 'Main':
            anime_cv_dict[name]['main'].append(c['cv_name'])
            name_count[c['cv_name']] += 1
            cv_count[c['cv_name']] += 1
        elif c['chara_type'] == 'Supporting':
            anime_cv_dict[name]['sup'].append(c['cv_name'])
            name_count[c['cv_name']] += 1
            cv_count[c['cv_name']] += 1
        else:
            continue
    st_info = d['staff']
    for s in st_info:
        anime_cv_dict[name]['staff'].append(s['staff_name'])
        name_count[s['staff_name']] += 1

HBox(children=(FloatProgress(value=0.0, max=9422.0), HTML(value='')))

In [14]:
len(name_count)

7501

In [16]:
name_set = [(n,c) for n,c in name_count.items() if c>1]
name_set.sort(key=lambda x:x[1], reverse=True)
print(name_set)

[('Hayashibara, Megumi', 281), ('Iwanami, Yoshikazu', 253), ('Sakurai, Takahiro', 238), ('Koyasu, Takehito', 232), ('Hanazawa, Kana', 229), ('Seki, Tomokazu', 227), ('Sawashiro, Miyuki', 226), ('Aketagawa, Jin', 226), ('Kugimiya, Rie', 225), ('Ishida, Akira', 216), ('Miki, Shinichiro', 213), ('Mima, Masafumi', 211), ('Horie, Yui', 209), ('Fukuyama, Jun', 205), ('Tsuruoka, Youta', 198), ('Namikawa, Daisuke', 192), ('Noto, Mamiko', 189), ('Morikawa, Toshiyuki', 185), ('Yamadera, Kouichi', 175), ('Kawasumi, Ayako', 175), ('Fujiwara, Keiji', 173), ('Kamiya, Hiroshi', 171), ('Yamaguchi, Kappei', 166), ('Miyano, Mamoru', 166), ('Ono, Daisuke', 165), ('Kaji, Yuki', 160), ('Inoue, Kikuko', 158), ('Inoue, Kazuhiko', 158), ('Kitamura, Eri', 155), ('Itou, Shizuka', 155), ('Ootani, Ikue', 154), ('Okiayu, Ryoutarou', 153), ('Mizuki, Nana', 152), ('Tsuru, Hiromi', 151), ('Saito, Chiwa', 150), ('Suwabe, Junichi', 150), ('Tanaka, Hideyuki', 149), ('Hisakawa, Aya', 146), ('Takayama, Minami', 145), ('No

In [17]:
len(name_set)

4646

In [19]:
cv_staff_name_idx = {}
for i,(n,_) in enumerate(name_set):
    cv_staff_name_idx[n] = i
with open('../archive/cv_staff_mapping.json','w') as w:
    json.dump(cv_staff_name_idx, w)

In [23]:
filter_anime_cv_dict = {}
# 过滤掉多余的工作人员和声优
for name in tqdm(anime_cv_dict):
    sp = {'main':[], 'sup':[], 'staff':[]}
    for m in anime_cv_dict[name]['main']:
        if m in cv_staff_name_idx:
            sp['main'].append(m)
    for s in anime_cv_dict[name]['sup']:
        if s in cv_staff_name_idx:
            sp['sup'].append(s)
    for s in anime_cv_dict[name]['staff']:
        if s in cv_staff_name_idx:
            sp['staff'].append(s)
    if len(sp['main'])+len(sp['sup'])+len(sp['staff'])>0:
        filter_anime_cv_dict[name] = sp
    
with open('../archive/cv_staff_data.json','w') as w:
    json.dump(filter_anime_cv_dict, w)

HBox(children=(FloatProgress(value=0.0, max=9422.0), HTML(value='')))

In [22]:
cv_set = [(n,c) for n,c in cv_count.items() if c>1]
cv_set.sort(key=lambda x:x[1], reverse=True)
print(cv_set)

[('Hayashibara, Megumi', 266), ('Sakurai, Takahiro', 237), ('Koyasu, Takehito', 229), ('Sawashiro, Miyuki', 222), ('Seki, Tomokazu', 221), ('Hanazawa, Kana', 221), ('Kugimiya, Rie', 218), ('Ishida, Akira', 215), ('Miki, Shinichiro', 207), ('Fukuyama, Jun', 201), ('Horie, Yui', 197), ('Namikawa, Daisuke', 187), ('Noto, Mamiko', 186), ('Morikawa, Toshiyuki', 181), ('Yamadera, Kouichi', 172), ('Kawasumi, Ayako', 171), ('Fujiwara, Keiji', 169), ('Kamiya, Hiroshi', 169), ('Yamaguchi, Kappei', 165), ('Ono, Daisuke', 160), ('Miyano, Mamoru', 159), ('Kaji, Yuki', 155), ('Ootani, Ikue', 153), ('Itou, Shizuka', 153), ('Okiayu, Ryoutarou', 153), ('Inoue, Kazuhiko', 153), ('Inoue, Kikuko', 152), ('Kitamura, Eri', 152), ('Tsuru, Hiromi', 150), ('Saito, Chiwa', 147), ('Suwabe, Junichi', 145), ('Hisakawa, Aya', 141), ('Takayama, Minami', 139), ('Nozawa, Masako', 139), ('Ohara, Sayaka', 138), ('Seki, Toshihiko', 137), ('Suzumura, Kenichi', 137), ('Midorikawa, Hikaru', 137), ('Nakamura, Yuuichi', 136),